In [3]:
import time
from tqdm import tqdm
from run_profiler import prerun_profiler, identify_attributes, get_attribute_function
from configs import  set_profiler_args

ModuleNotFoundError: No module named 'utils'

# Set Up

In [ ]:
profiler_args= set_profiler_args({
    "data_lake": "wiki_nba", 
    "num_attr_to_cascade": 50, 
    "num_top_k_scripts": 10, 
    "train_size": 10, 
    "combiner_mode": "mv", 
    "do_end_to_end": False,
    "use_dynamic_backoff": True, 
    "KEYS": [""], 
    "MODELS":["mistralai/Mistral-7B-instruct-v0.1", "gpt-4"],
    "EXTRACTION_MODELS": ["mistralai/Mistral-7B-instruct-v0.1"],
    "GOLD_KEY": "gpt-4",
    "MODEL2URL" : {"mistralai/Mistral-7B-instruct-v0.1": "http://127.0.0.1:5000"},
    "data_dir": "/data/evaporate/data/wiki_nba_players/home/simran/fm-data-analysis/data", 
    "base_data_dir": "/data/evaporate/data/wiki_nba_players", 
    "gold_extractions_file": "/data/evaporate/data/wiki_nba_players/table.json"}
)
print(profiler_args)

In [ ]:
data_dict = prerun_profiler(profiler_args)
print(data_dict.keys())

In [ ]:
print(profiler_args.overwrite_cache)

# Get Schema Attribute

In [ ]:
if  profiler_args.do_end_to_end:
    attributes, total_time, num_toks, evaluation_result = identify_attributes(profiler_args, data_dict, evaluation=True)
    print("total_time: ", total_time, "num_toks: ", num_toks, "evaluation_result: ", evaluation_result)
else:
    attributes = data_dict["gold_attributes"]

In [ ]:
attributes

Model Information
Data Dir
CONSTANTS = {
        "fda_510ks": {
            "data_dir": os.path.join(BASE_DATA_DIR, "fda-ai-pmas/510k/"),
            "database_name": "fda_510ks",
            "cache_dir": ".cache/fda_510ks/",
            "generative_index_path": os.path.join(BASE_DATA_DIR, "generative_indexes/fda_510ks/"),
            "gold_extractions_file": os.path.join(BASE_DATA_DIR, "ground_truth/fda_510ks_gold_extractions.json"),
            "topic": "fda 510k device premarket notifications",
        },
    }

# Get Attribute function

In [ ]:
functions = {}
selected_keys = {}
for i, attribute in enumerate(attributes):
    print(f"\n\nExtracting {attribute} ({i+1} / {len(attributes)})")
    t0 = time.time()
    functions[attribute], selected_keys[attribute], total_time, num_toks= get_attribute_function(
        profiler_args, data_dict, attribute
    )
    print(functions[attribute])

# Evaluation

In [ ]:
from evaporate.evaluate_synthetic import main as evaluate_synthetic_main

In [ ]:

results = evaluate_synthetic_main(
    profiler_args.run_string, 
    profiler_args, 
    profiler_args, 
    profiler_args.data_lake,
    gold_attributes=data_dict["gold_attributes"], 
    stage='extract'
)
print(results)